In [39]:
import os
from openpyxl import Workbook
from datetime import datetime
import pandas as pd

In [40]:
user_input = 'prism_1n_le20_basic.log' # Enter path to log here
assert os.path.exists(user_input), "I did not find the file at "+str(user_input)
f = open(user_input,'r')
print("Log File found, beginning analysis...")
f.close()

Log File found, beginning analysis...


In [41]:
ProposerBlockMined = 'ProposerBlockMined'
ValidProposerBlockDownloaded = 'ValidProposerBlockDownloaded'
ProposerBlockConfirmed = 'ProposerBlockConfirmed'
ProposerBlockRolledBack = 'ProposerBlockRolledBack'
epoch = datetime.utcfromtimestamp(0)

hashToNumber = {}
hashToCreationTimestamp = {}
hashToCreationEvent = {}
hashToConfirmationTimestamp = {}
hashToRollbackTimestamp = {}

In [42]:
def getTimestamp(timestr):
    dateTimeObj = datetime.strptime(timestr, '%Y-%m-%d %H:%M:%S,%f')
    delta = dateTimeObj - epoch
    return delta.total_seconds()*1000

In [43]:
with open(user_input, 'r') as f:
    for line in f:
        if "BENCHMARKING:" not in line:
            continue
        benchmarkSubstrings = line[line.find("BENCHMARKING"):-1].split(':')
        dateTimeString = line.split('[')[0][:-1]
        timestamp = getTimestamp(dateTimeString)
        event = benchmarkSubstrings[1]
        blockNumber = benchmarkSubstrings[3]
        blockHash = benchmarkSubstrings[-1][1:-1]
        hashToNumber[blockHash] = blockNumber
        if event == ProposerBlockMined or event == ValidProposerBlockDownloaded:
            hashToCreationEvent[blockHash] = event
            hashToCreationTimestamp[blockHash] = timestamp
        if event == ProposerBlockConfirmed:
            hashToConfirmationTimestamp[blockHash] = timestamp
        if event == ProposerBlockRolledBack:
            hashToRollbackTimestamp[blockHash] = timestamp

f.close()

In [48]:
df = pd.DataFrame(columns=['Block Hash','Block Number','Timestamp - Creation','Event','Timestamp - Confirmation','Timestamp - Rollback'])

for index,key in enumerate(hashToNumber.keys()):
    df.loc[index] = [key,
        int(hashToNumber.get(key,'')),
        hashToCreationTimestamp.get(key,''),
        hashToCreationEvent.get(key,''),
        hashToConfirmationTimestamp.get(key,''),
        hashToRollbackTimestamp.get(key,'')]

df.sort_values('Block Number')

,Block Hash,Block Number,Timestamp - Creation,Event,Timestamp - Confirmation,Timestamp - Rollback
0,0xc659242f21b99e43764659c4c1ba24988871f2e6ae20...,1,1.638951e+12,ProposerBlockMined,1638952850170.0,
1,0x99bda9d804ec67ec6c0b7512e963d3d2c2971dc3b63e...,2,1.638951e+12,ProposerBlockMined,1638952861732.0,
2,0x6aebbe9a1eb3cb2fa2873e34a2d7f74f0a4752027c7a...,3,1.638951e+12,ProposerBlockMined,1638952874343.0,
3,0xde3794ea19fdd5f8433e20c04c8194f9471164567572...,4,1.638951e+12,ProposerBlockMined,1638952875366.0,
4,0xfaa87f0c4bd1d64abc199d44d52928ca1f44094359b5...,5,1.638951e+12,ProposerBlockMined,1638952877772.0,
...,...,...,...,...,...,...
132,0xe43927b3c937a9d7a1395c6706011eba157d9460a050...,133,1.638957e+12,ProposerBlockMined,,
133,0x7413c6e8f72ebb4cf5e80c12f89030a9c1a1e0790a1a...,134,1.638957e+12,ProposerBlockMined,,
134,0x7b6a4b113e1643ab5ed2b592adf8d3c52cd7e1d81f0c...,135,1.638957e+12,ProposerBlockMined,,
135,0xd0714b43ad7edae4bb4993de577dae5255747de237cd...,136,1.638957e+12,ProposerBlockMined,,


In [49]:
df['Latency'] = df[df['Timestamp - Confirmation']!='']['Timestamp - Confirmation']-df[df['Timestamp - Confirmation']!='']['Timestamp - Creation']
df['Mining Time'] = df['Timestamp - Creation'].diff()
df['Confirmation Depth']=df['Latency']/df['Mining Time']

In [50]:
print(df['Latency'].dropna().mean())
print(df['Latency'].dropna().median())
print(df['Mining Time'].dropna().mean())
print(df['Mining Time'].dropna().median())
print(df['Confirmation Depth'].dropna().mean())
print(df['Confirmation Depth'].dropna().median())

1446224.5
1455759.5
43546.007352941175
26200.0
400.8339703616673
53.429270430146794
